on va tenter de scrapp le wiki fandom 

In [11]:
from requests import get
import requests
from bs4 import BeautifulSoup
import re
import time
import polars as pl
from sklearn.model_selection import train_test_split

KeyboardInterrupt: 

In [ ]:
url = "https://leagueoflegends.fandom.com/wiki/{}/LoL"
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:146.0) Gecko/20100101 Firefox/146.0'
}
page = get(url=url, headers=headers)
page.raise_for_status() 
print("Success!")
page.ok

In [10]:
html_brut = page.text # 2. On initialise le parser
soup = BeautifulSoup(html_brut, 'html.parser')

In [11]:
soup
soup.title

<title>Aatrox (League of Legends) | League of Legends Wiki | Fandom</title>

On va essayer d'avoir tout les noms de champ d'un coup

In [3]:
url = "https://leagueoflegends.fandom.com/wiki/Champion#List_of_champions"
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:146.0) Gecko/20100101 Firefox/146.0'
}
page_champ = get(url=url, headers=headers)
page_champ.raise_for_status() 
print("Success!")
page_champ.ok

Success!


True

On crée le parser

In [4]:
html_brut_page_champ = page_champ.text
soup_page_champ = BeautifulSoup(html_brut_page_champ, 'html.parser')

In [ ]:
soup_page_champ

#Récupération de data dragon riot

In [6]:
# 1. Télécharger les données
url = "https://ddragon.leagueoflegends.com/cdn/15.24.1/data/en_US/champion.json"
response = requests.get(url)
data = response.json()

# 2. Les champions
champions_dict = data['data']

# 3On chope les infos
rows = []
for champ_name, details in champions_dict.items():
    stats = details['stats']
    info = details['info']
    rows.append({
        "Name": champ_name,
        "Class": details['tags'][0],  # On prend le rôle principal pour la classification
        "AttackRange": stats['attackrange'],
        "MoveSpeed": stats['movespeed'],

        "HP": stats['hp'],
        "HPperLevel": stats['hpperlevel'],

        "hpregen": stats['hpregen'],
        "hpregenperlevel": stats['hpregenperlevel'],

        "Armor": stats['armor'],
        "ArmorperLevel": stats['armorperlevel'],
        
        "AttackDamage": stats['attackdamage'],
        "AttackDamageperLevel": stats['attackdamageperlevel'],

        "attackspeed": stats['attackspeed'],
        "attackspeedperlevel": stats['attackspeedperlevel'],

        "Spellblock": stats['spellblock'],
        "spellblockperlevel": stats['spellblockperlevel'],

        "Prv_attack": info['attack'],
        "Prv_defense": info['defense'],
        "Prv_magic": info['magic']
    })

# 4. Créer le Tableau
tableau_champ = pl.DataFrame(rows)

print(tableau_champ)

shape: (172, 19)
┌─────────┬──────────┬────────────┬───────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ Name    ┆ Class    ┆ AttackRang ┆ MoveSpeed ┆ … ┆ spellblock ┆ Prv_attac ┆ Prv_defen ┆ Prv_magic │
│ ---     ┆ ---      ┆ e          ┆ ---       ┆   ┆ perlevel   ┆ k         ┆ se        ┆ ---       │
│ str     ┆ str      ┆ ---        ┆ i64       ┆   ┆ ---        ┆ ---       ┆ ---       ┆ i64       │
│         ┆          ┆ i64        ┆           ┆   ┆ f64        ┆ i64       ┆ i64       ┆           │
╞═════════╪══════════╪════════════╪═══════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
│ Aatrox  ┆ Fighter  ┆ 175        ┆ 345       ┆ … ┆ 2.05       ┆ 8         ┆ 4         ┆ 3         │
│ Ahri    ┆ Mage     ┆ 550        ┆ 330       ┆ … ┆ 1.3        ┆ 3         ┆ 4         ┆ 8         │
│ Akali   ┆ Assassin ┆ 125        ┆ 345       ┆ … ┆ 2.05       ┆ 5         ┆ 3         ┆ 8         │
│ Akshan  ┆ Marksman ┆ 500        ┆ 330       ┆ … ┆ 1.3        ┆ 0        

#Transformation du tableau pour classification 

In [7]:
# 1. On définit la correspondance entre les noms de classes et les chiffres
# (C'est ce qu'on appelle le Label Encoding)
classes_uniques = tableau_champ["Class"].unique().to_list()
mapping_classes = {nom: i for i, nom in enumerate(classes_uniques)}

# 2. On crée le tableau de classification
# On enlève le "Name" (inutile pour prédire) et on convertit la "Class" en nombre
tableau_ml = tableau_champ.with_columns(
    pl.col("Class").replace(mapping_classes).cast(pl.Int64).alias("Target")
).drop("Name")

# 3. Affichage pour vérification
print("Dictionnaire de correspondance (pour ton rapport) :")
print(mapping_classes)
print("\nTableau prêt pour l'IA (Aperçu) :")
print(tableau_ml.head())

Dictionnaire de correspondance (pour ton rapport) :
{'Marksman': 0, 'Assassin': 1, 'Fighter': 2, 'Support': 3, 'Tank': 4, 'Mage': 5}

Tableau prêt pour l'IA (Aperçu) :
shape: (5, 19)
┌──────────┬─────────────┬───────────┬─────┬───┬────────────┬─────────────┬───────────┬────────┐
│ Class    ┆ AttackRange ┆ MoveSpeed ┆ HP  ┆ … ┆ Prv_attack ┆ Prv_defense ┆ Prv_magic ┆ Target │
│ ---      ┆ ---         ┆ ---       ┆ --- ┆   ┆ ---        ┆ ---         ┆ ---       ┆ ---    │
│ str      ┆ i64         ┆ i64       ┆ i64 ┆   ┆ i64        ┆ i64         ┆ i64       ┆ i64    │
╞══════════╪═════════════╪═══════════╪═════╪═══╪════════════╪═════════════╪═══════════╪════════╡
│ Fighter  ┆ 175         ┆ 345       ┆ 650 ┆ … ┆ 8          ┆ 4           ┆ 3         ┆ 2      │
│ Mage     ┆ 550         ┆ 330       ┆ 590 ┆ … ┆ 3          ┆ 4           ┆ 8         ┆ 5      │
│ Assassin ┆ 125         ┆ 345       ┆ 600 ┆ … ┆ 5          ┆ 3           ┆ 8         ┆ 1      │
│ Marksman ┆ 500         ┆ 330       ┆ 61

#Vérification 

In [8]:
# Vérifier le type de chaque colonne
print("Types des colonnes dans tableau_ml :")
print(tableau_ml.schema)

# Vérifier s'il reste des colonnes de type 'String' (Objet/Texte)
colonnes_texte = [name for name, dtype in tableau_ml.schema.items() if dtype == pl.String]

if len(colonnes_texte) > 0:
    print(f"\n⚠️ Attention ! Il reste du texte dans : {colonnes_texte}")
else:
    print("\n✅ Parfait ! Toutes les colonnes sont numériques. Ton IA peut travailler.")

Types des colonnes dans tableau_ml :
Schema({'Class': String, 'AttackRange': Int64, 'MoveSpeed': Int64, 'HP': Int64, 'HPperLevel': Int64, 'hpregen': Float64, 'hpregenperlevel': Float64, 'Armor': Int64, 'ArmorperLevel': Float64, 'AttackDamage': Int64, 'AttackDamageperLevel': Float64, 'attackspeed': Float64, 'attackspeedperlevel': Float64, 'Spellblock': Int64, 'spellblockperlevel': Float64, 'Prv_attack': Int64, 'Prv_defense': Int64, 'Prv_magic': Int64, 'Target': Int64})

⚠️ Attention ! Il reste du texte dans : ['Class']


#Séparation

In [10]:
# 1. On définit la liste des colonnes statistiques (les "Features")
# On exclut "Name", "Class" et "Target"
features_cols = [
    col for col in tableau_ml.columns 
    if col not in ["Name", "Class", "Target"]
]

# 2. On crée X (les données pour l'IA) et y (la réponse attendue)
X = tableau_ml.select(features_cols)
y = tableau_ml.select("Target")

print(f"L'IA va s'entraîner sur {len(features_cols)} caractéristiques.")
print(f"Colonnes utilisées pour X : {features_cols}")

L'IA va s'entraîner sur 17 caractéristiques.
Colonnes utilisées pour X : ['AttackRange', 'MoveSpeed', 'HP', 'HPperLevel', 'hpregen', 'hpregenperlevel', 'Armor', 'ArmorperLevel', 'AttackDamage', 'AttackDamageperLevel', 'attackspeed', 'attackspeedperlevel', 'Spellblock', 'spellblockperlevel', 'Prv_attack', 'Prv_defense', 'Prv_magic']


#Groupes d'entrainements